In [ ]:
from scripts.StreamProcessing import Stream_Data # found in the scripts folder inside StreamProcessing.py. This class uses PySpark

https://berkeleyearth.org/data/

### A note on block 

These might take a while as PySpark attempts to download some additional files

In [ ]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
emissions_config = Stream_Data(topics='ghg_data', host='localhost:9092')
e_session, query, emissions_df = emissions_config.getData()
#temperatures

In [ ]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
temperature_config = Stream_Data(topics='temperature', host='localhost:9092')
t_session, query_temperature, temperature_df = temperature_config.getData()

### NOTE
Everything is configered while initializing the PySpark Session in the ```StreamProcessing.py``` class. DO NOT remove the ```from graphframes import GraphFrame```, as the jar files are loaded while getting the data from the previous code block

In [ ]:
from graphframes import GraphFrame
import findspark as tt
tt.init(emissions_config.getSpark_Path()) 
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:
print(temperature_df)
print(emissions_df)

In [ ]:
temp_df = temperature_df.withColumnRenamed('REF_CODE','country_code')
ghg_df = emissions_df.withColumnRenamed('PREFIX', 'country_code')
joined_dataset = temp_df.join(ghg_df, 'country_code')



# ran into out of memory exception

# We'll be assigning the country_codes as ids to map relationships